In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [2]:
engine = create_engine("postgresql:///Seattle_housing")

In [3]:
# query1 = """SELECT 
#                CONCAT(rp."Major", rp."Minor") AS pin,
#                "SalePrice", 
#                "SqFtTotLiving" AS SquareFeet, 
#                CASE 
#                    WHEN ("SqFtOpenPorch" + "SqFtEnclosedPorch" > 0) THEN 1
#                    ELSE 0 
#                    END AS porch,
#                CASE 
#                    WHEN "WfntLocation" > 0 THEN 1
#                    ELSE 0
#                    END AS WaterFront,
#                (("SqFt1stFloor" + "SqFtGarageAttached") / "SqFtLot" ) AS proportion,
#                CASE 
#                    WHEN ("TrafficNoise" + "AirportNoise" > 0) OR ("PowerLines" = 'Y') OR ("OtherNuisances" = 'Y') THEN 1
#                    ELSE 0
#                    END AS nuisance
#             FROM rpsale rp
#             JOIN resbldg rd
#             ON CONCAT(rp."Major", rp."Minor") = Concat(rd."Major", rd."Minor")
#             JOIN parcel p
#             ON CONCAT(rp."Major", rp."Minor") = Concat(p."Major", p."Minor")
#             WHERE "SalePrice" > 0
#             AND SUBSTRING("DocumentDate", 7, 4) = '2018'
#         """

In [4]:
# df1 = pd.read_sql(sql = query1, con = engine)

In [5]:
query2 = """SELECT 
               *
            FROM rpsale rp
            JOIN resbldg rd
            ON CONCAT(rp."Major", rp."Minor") = Concat(rd."Major", rd."Minor")
            JOIN parcel p
            ON CONCAT(rp."Major", rp."Minor") = Concat(p."Major", p."Minor")
            WHERE "SalePrice" > 5000
            AND SUBSTRING("DocumentDate", 7, 4) = '2018'
         """

In [6]:
seattle_housing = pd.read_sql(sql = query2, con = engine)

In [7]:
seattle_housing.columns

Index(['ExciseTaxNbr', 'Major', 'Minor', 'DocumentDate', 'SalePrice',
       'RecordingNbr', 'Volume', 'Page', 'PlatNbr', 'PlatType',
       ...
       'SeismicHazard', 'LandslideHazard', 'SteepSlopeHazard', 'Stream',
       'Wetland', 'SpeciesOfConcern', 'SensitiveAreaTract', 'WaterProblems',
       'TranspConcurrency', 'OtherProblems'],
      dtype='object', length=155)

In [8]:
seattle_housing.describe()

,ExciseTaxNbr,SalePrice,PropertyType,PrincipalUse,SaleInstrument,SaleReason,PropertyClass,BldgNbr,NbrLivingUnits,Stories,...,WfntPoorQuality,WfntRestrictedAccess,TidelandShoreland,LotDepthFactor,TrafficNoise,AirportNoise,NbrBldgSites,Contamination,HistoricSite,CurrentUseDesignation
count,3.000300e+04,3.000300e+04,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,...,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.0,30003.000000,30003.0,30003.000000,30003.000000
mean,2.939962e+06,8.314959e+05,7.238976,6.000567,3.773523,2.004066,7.959971,1.020498,1.019098,1.585241,...,0.000900,0.006333,0.016232,0.026464,0.208446,0.0,0.016865,0.0,0.001267,0.005733
std,1.681032e+04,7.795134e+05,6.632176,0.099494,3.618102,3.891312,0.279450,0.254314,0.159165,0.561490,...,0.029985,0.117866,0.177673,1.394980,0.562191,0.0,0.400547,0.0,0.061086,0.124497
min,2.909224e+06,5.200000e+03,0.000000,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
25%,2.925868e+06,4.499500e+05,3.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
50%,2.939709e+06,6.650000e+05,10.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
75%,2.954250e+06,9.400000e+05,11.000000,6.000000,3.000000,1.000000,8.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
max,3.017086e+06,2.675000e+07,99.000000,10.000000,28.000000,19.000000,9.000000,12.000000,3.000000,4.000000,...,1.000000,3.000000,2.000000,96.000000,3.000000,0.0,20.000000,0.0,3.000000,4.000000


In [9]:
# delcolumns = ["PropName", "PlatName", "PlatLot", "PlatBlock", "Range",
#               "Township", "Section", "QuarterSection", "Area", "SubArea",
#               "SpecArea", "SpecSubArea", "ExciseTaxNbr", "RecordingNbr",
#               "Volume", "Page", "PlatNbr", "PlatType",
#               "SellerName", "BuyerName", "BldgNbr", "Address", "BuildingNumber",
#               "Fraction", "DirectionPrefix", "StreetName", "StreetType", 
#               "DirectionSuffix", "AirportNoise", "Contamination"]

In [10]:
# for column in delcolumns:
#     del seattle_housing[column]

In [11]:
#Remove house sales less than $5000
#seattle_housing = seattle_housing[seattle_housing['SalePrice'] >= 5000]

In [12]:
seattle_housing.shape

(30003, 155)

In [13]:
keepcol = ['SalePrice', 'SqFtTotLiving', 'SqFtLot', 'Bedrooms', 'BathHalfCount', 'Bath3qtrCount',
           'BathFullCount', 'YrBuilt', 'SqFtGarageBasement', 'SqFtGarageAttached', 'Area']

In [14]:
model1df = seattle_housing[keepcol]

In [15]:
#Make total number of bathrooms column - maybe consider multiplying by 0.5/0.75
model1df['Bathrooms'] = model1df['BathHalfCount'] + model1df['Bath3qtrCount'] + model1df['BathFullCount']

//anaconda3/envs/linreg-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
model1df['AgeOfBuilding'] = 2018 - model1df['YrBuilt']

//anaconda3/envs/linreg-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
model1df['Garage'] = model1df['SqFtGarageAttached'] + model1df['SqFtGarageBasement']

//anaconda3/envs/linreg-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
model1df.shape

(30003, 14)

In [19]:
ohe = OneHotEncoder(categories='auto')

model1df_trans = ohe.fit_transform(model1df[['Area']])

In [20]:
model1df_trans.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]])

In [21]:
df = pd.DataFrame(model1df_trans.todense(), columns = ohe.get_feature_names())
df.shape

(30003, 88)

In [22]:
model1_edit = pd.concat([model1df, df], axis=1)

In [23]:
model1_edit.shape


(30003, 102)

In [24]:
model1_edit = model1_edit.drop(['BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'YrBuilt', 'SqFtGarageBasement', 
                  'SqFtGarageAttached', 'Area', 'x0_86 '], axis = 1)






In [25]:
features1 = model1_edit.iloc[:, 1:]
dep_variable = model1_edit['SalePrice']

In [26]:
model = sm.OLS(dep_variable, sm.add_constant(features1))
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.430
Method:                 Least Squares   F-statistic:                     244.4
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:02:47   Log-Likelihood:            -4.4112e+05
No. Observations:               30003   AIC:                         8.824e+05
Df Residuals:                   29909   BIC:                         8.832e+05
Df Model:                          93                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1.049e+04   3.13e+04     -0.335      0.737   -7.18e+04    5.08e+04
SqFtTotLiving   363.5514      6.277     57.921      0.000     351.249     375.854
SqFtLot           0.3654      0.066      5.509      0.000       0.235       0.495
Bedrooms      -6.641e+04   4689.438    -14.162      0.000   -7.56e+04   -5.72e+04
Bathrooms      9953.1976   5975.599      1.666      0.096   -1759.236    2.17e+04
AgeOfBuilding -1084.3833    145.795     -7.438      0.000   -1370.148    -798.618
Garage         -128.6858     16.560     -7.771      0.000    -161.144     -96.227
x0_1           3.406e+05   4.46e+04      7.638      0.000    2.53e+05    4.28e+05
x0_100         2.163e+05   5.09e+04      4.253      0.000    1.17e+05    3.16e+05
x0_11          6.358e+05   4.11e+04     15.460      0.000    5.55e+05    7.16e+05
x0_12          8.246e+05   4.05e+04     20.350      0.000    7.45e+05    9.04e+05
x0_13          8.711e+05   4.17e+04     20.881      0.000    7.89e+05    9.53e+05
x0_14          1.052e+06   4.96e+04     21.190      0.000    9.55e+05    1.15e+06
x0_15          5.145e+05   3.84e+04     13.404      0.000    4.39e+05     5.9e+05
x0_16           5.03e+05   4.15e+04     12.116      0.000    4.22e+05    5.84e+05
x0_17           3.42e+05   4.19e+04      8.165      0.000     2.6e+05    4.24e+05
x0_18          2.843e+05   3.93e+04      7.241      0.000    2.07e+05    3.61e+05
x0_19          5.261e+05   4.05e+04     13.002      0.000    4.47e+05    6.05e+05
x0_2           3.194e+05   4.97e+04      6.421      0.000    2.22e+05    4.17e+05
x0_21          2.946e+05      4e+04      7.366      0.000    2.16e+05    3.73e+05
x0_22          1.534e+05   4.86e+04      3.154      0.002    5.81e+04    2.49e+05
x0_23           1.12e+05   3.92e+04      2.853      0.004     3.5e+04    1.89e+05
x0_24          6.431e+04   3.91e+04      1.646      0.100   -1.23e+04    1.41e+05
x0_25          1.357e+05   4.11e+04      3.300      0.001    5.51e+04    2.16e+05
x0_26          5953.1654   4.04e+04      0.147      0.883   -7.32e+04    8.51e+04
x0_27          5.716e+05   3.94e+04     14.493      0.000    4.94e+05    6.49e+05
x0_28          5.643e+04   3.86e+04      1.461      0.144   -1.93e+04    1.32e+05
x0_29          4.815e+04   4.49e+04      1.073      0.283   -3.98e+04    1.36e+05
x0_3           2.631e+05   4.45e+04      5.917      0.000    1.76e+05     3.5e+05
x0_30          8.582e+04   3.98e+04      2.154      0.031    7719.321    1.64e+05
x0_31          4.791e+05   4.29e+04     11.172      0.000    3.95e+05    5.63e+05
x0_32          1.882e+05   3.97e+04      4.746      0.000     1.1e+05    2.66e+05
x0_33          2.445e+06   4.98e+04     49.100      0.000    2.35e+06    2.54e+06
x0_34          1.326e+06   4.16e+04     31.869      0.000    1.24e+06    1.41e+06
x0_35          3.272e+05   3.57e+04      9.156   

In [27]:
#Try looking for normal distributions of data
#Try normalizing or log transforming
#Try removing strongly colinear variables?

In [28]:
features2 = model1_edit.drop(['SalePrice', 'Bathrooms', 'x0_2  ', 'x0_3  ', 'x0_4  ', 'x0_18 ', 'x0_21 ',
                              'x0_22 ', 'x0_25 ', 'x0_32 ', 'x0_36 ', 'x0_38 ', 'x0_41 ', 'x0_49 ',
                              'x0_51 ', 'x0_53 ', 'x0_57 ', 'x0_60 ', 'x0_61 ', 'x0_69 ', 'x0_71 ', 'x0_73 ',
                              'x0_75 ', 'x0_77 ', 'x0_85 ', 'x0_90 ', 'x0_95 ', 'x0_96 '], axis = 1)

In [29]:
model2 = sm.OLS(dep_variable, sm.add_constant(features2))
result2 = model2.fit()
result2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.428
Model:                            OLS   Adj. R-squared:                  0.427
Method:                 Least Squares   F-statistic:                     339.9
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:02:47   Log-Likelihood:            -4.4122e+05
No. Observations:               30003   AIC:                         8.826e+05
Df Residuals:                   29936   BIC:                         8.831e+05
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2.154e+05   1.62e+04     13.273      0.000    1.84e+05    2.47e+05
SqFtTotLiving   375.9013      5.271     71.314      0.000     365.570     386.233
SqFtLot           0.3693      0.066      5.617      0.000       0.240       0.498
Bedrooms      -6.841e+04   4560.113    -15.002      0.000   -7.73e+04   -5.95e+04
AgeOfBuilding -1230.6451    131.785     -9.338      0.000   -1488.949    -972.341
Garage         -130.2047     16.138     -8.068      0.000    -161.835     -98.574
x0_1           1.267e+05   3.62e+04      3.502      0.000    5.58e+04    1.98e+05
x0_100         2281.1916   4.36e+04      0.052      0.958   -8.32e+04    8.77e+04
x0_11          4.229e+05   3.16e+04     13.395      0.000    3.61e+05    4.85e+05
x0_12          6.147e+05   3.06e+04     20.062      0.000    5.55e+05    6.75e+05
x0_13          6.607e+05   3.22e+04     20.522      0.000    5.98e+05    7.24e+05
x0_14          8.398e+05   4.19e+04     20.064      0.000    7.58e+05    9.22e+05
x0_15           3.05e+05    2.8e+04     10.897      0.000     2.5e+05     3.6e+05
x0_16          2.917e+05   3.21e+04      9.090      0.000    2.29e+05    3.55e+05
x0_17          1.296e+05   3.27e+04      3.968      0.000    6.56e+04    1.94e+05
x0_19          3.162e+05   3.08e+04     10.268      0.000    2.56e+05    3.77e+05
x0_23          -9.93e+04   2.93e+04     -3.388      0.001   -1.57e+05   -4.19e+04
x0_24         -1.471e+05   2.91e+04     -5.057      0.000   -2.04e+05   -9.01e+04
x0_26         -2.063e+05    3.1e+04     -6.662      0.000   -2.67e+05   -1.46e+05
x0_27          3.565e+05   2.98e+04     11.969      0.000    2.98e+05    4.15e+05
x0_28         -1.544e+05   2.85e+04     -5.416      0.000    -2.1e+05   -9.86e+04
x0_29          -1.65e+05   3.67e+04     -4.496      0.000   -2.37e+05   -9.31e+04
x0_30         -1.259e+05   3.03e+04     -4.159      0.000   -1.85e+05   -6.66e+04
x0_31          2.644e+05   3.41e+04      7.757      0.000    1.98e+05    3.31e+05
x0_33          2.226e+06   4.22e+04     52.750      0.000    2.14e+06    2.31e+06
x0_34          1.109e+06   3.23e+04     34.354      0.000    1.05e+06    1.17e+06
x0_35          1.104e+05   2.45e+04      4.504      0.000    6.24e+04    1.58e+05
x0_37          1.705e+05   2.69e+04      6.341      0.000    1.18e+05    2.23e+05
x0_39          2.895e+05   3.37e+04      8.597      0.000    2.23e+05    3.55e+05
x0_40           -2.6e+05   4.39e+04     -5.928      0.000   -3.46e+05   -1.74e+05
x0_42           4.09e+05   3.48e+04     11.767      0.000    3.41e+05    4.77e+05
x0_43          4.376e+05   3.19e+04     13.704      0.000    3.75e+05       5e+05
x0_44          3.605e+05   4.05e+04      8.901      0.000    2.81e+05     4.4e+05
x0_45          3.066e+05   3.59e+04      8.535      0.000    2.36e+05    3.77e+05
x0_46          9.163e+05   4.42e+04     20.716   

In [30]:
features2.corr()

,SqFtTotLiving,SqFtLot,Bedrooms,AgeOfBuilding,Garage,x0_1,x0_100,x0_11,x0_12,x0_13,...,x0_80,x0_81,x0_82,x0_84,x0_87,x0_88,x0_91,x0_92,x0_93,x0_94
SqFtTotLiving,1.000000,0.114795,0.626451,-0.374174,0.523429,0.014841,-0.028488,0.024588,-0.007112,0.008224,...,-0.010943,-0.007568,-0.066595,0.005654,-0.021405,-0.050566,0.008497,0.112972,0.022974,0.001950
SqFtLot,0.114795,1.000000,-0.005946,0.002427,0.077089,0.001138,0.090089,-0.022579,-0.027517,-0.024695,...,0.055422,-0.014820,-0.025293,0.077981,-0.003262,-0.005175,-0.010410,-0.005511,-0.014219,0.117905
Bedrooms,0.626451,-0.005946,1.000000,-0.224278,0.321461,0.007376,-0.062154,-0.005111,-0.035157,-0.012246,...,-0.019814,-0.007744,-0.062739,-0.019447,-0.007216,-0.020216,0.025595,0.054780,0.024959,-0.011530
AgeOfBuilding,-0.374174,0.002427,-0.224278,1.000000,-0.496488,0.037059,0.029794,0.050313,0.058539,0.075944,...,-0.041007,0.070910,0.086716,0.004987,-0.036670,0.024416,0.006069,-0.021550,-0.042971,-0.014826
Garage,0.523429,0.077089,0.321461,-0.496488,1.000000,0.005315,-0.049562,-0.047235,-0.077971,-0.079412,...,0.018834,-0.053100,-0.092109,0.020474,0.015169,-0.013834,0.038192,0.053686,0.038835,0.014546
x0_1,0.014841,0.001138,0.007376,0.037059,0.005315,1.000000,-0.007678,-0.010718,-0.011081,-0.010539,...,-0.012559,-0.008094,-0.010312,-0.007303,-0.008709,-0.011152,-0.007958,-0.008114,-0.011138,-0.007596
x0_100,-0.028488,0.090089,-0.062154,0.029794,-0.049562,-0.007678,1.000000,-0.008896,-0.009197,-0.008748,...,-0.010424,-0.006718,-0.008560,-0.006062,-0.007229,-0.009256,-0.006605,-0.006734,-0.009244,-0.006305
x0_11,0.024588,-0.022579,-0.005111,0.050313,-0.047235,-0.010718,-0.008896,1.000000,-0.012838,-0.012211,...,-0.014551,-0.009378,-0.011948,-0.008462,-0.010090,-0.012921,-0.009220,-0.009400,-0.012904,-0.008801
x0_12,-0.007112,-0.027517,-0.035157,0.058539,-0.077971,-0.011081,-0.009197,-0.012838,1.000000,-0.012624,...,-0.015044,-0.009695,-0.012352,-0.008748,-0.010432,-0.013358,-0.009532,-0.009719,-0.013341,-0.009099
x0_13,0.008224,-0.024695,-0.012246,0.075944,-0.079412,-0.010539,-0.008748,-0.012211,-0.012624,1.000000,...,-0.014309,-0.009222,-0.011749,-0.008321,-0.009922,-0.012706,-0.009067,-0.009244,-0.012689,-0.008655


In [31]:
ss = StandardScaler()

features2_skscaled = ss.fit_transform(features2)

In [32]:
model3 = sm.OLS(dep_variable, sm.add_constant(features2_skscaled))
result3 = model3.fit()
result3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.428
Model:                            OLS   Adj. R-squared:                  0.427
Method:                 Least Squares   F-statistic:                     339.9
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:02:48   Log-Likelihood:            -4.4122e+05
No. Observations:               30003   AIC:                         8.826e+05
Df Residuals:                   29936   BIC:                         8.831e+05
Df Model:                          66                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.315e+05   3406.303    244.105      0.000    8.25e+05    8.38e+05
x1          3.789e+05   5312.818     71.314      0.000    3.68e+05    3.89e+05
x2          2.116e+04   3767.522      5.617      0.000    1.38e+04    2.85e+04
x3          -6.73e+04   4486.428    -15.002      0.000   -7.61e+04   -5.85e+04
x4         -3.919e+04   4197.103     -9.338      0.000   -4.74e+04    -3.1e+04
x5         -3.699e+04   4584.208     -8.068      0.000    -4.6e+04    -2.8e+04
x6          1.207e+04   3446.320      3.502      0.000    5315.139    1.88e+04
x7           180.9576   3457.870      0.052      0.958   -6596.618    6958.533
x8          4.655e+04   3475.122     13.395      0.000    3.97e+04    5.34e+04
x9          6.989e+04   3483.494     20.062      0.000    6.31e+04    7.67e+04
x10         7.154e+04   3486.270     20.522      0.000    6.47e+04    7.84e+04
x11         6.985e+04   3481.211     20.064      0.000     6.3e+04    7.67e+04
x12         3.811e+04   3496.951     10.897      0.000    3.13e+04     4.5e+04
x13         3.159e+04   3474.815      9.090      0.000    2.48e+04    3.84e+04
x14         1.376e+04   3466.278      3.968      0.000    6961.832    2.05e+04
x15         3.568e+04   3474.476     10.268      0.000    2.89e+04    4.25e+04
x16        -1.176e+04   3471.196     -3.388      0.001   -1.86e+04   -4958.420
x17        -1.753e+04   3466.265     -5.057      0.000   -2.43e+04   -1.07e+04
x18          -2.3e+04   3453.119     -6.662      0.000   -2.98e+04   -1.62e+04
x19         4.139e+04   3457.754     11.969      0.000    3.46e+04    4.82e+04
x20        -1.883e+04   3477.401     -5.416      0.000   -2.56e+04    -1.2e+04
x21        -1.547e+04   3440.466     -4.496      0.000   -2.22e+04   -8725.541
x22        -1.438e+04   3458.908     -4.159      0.000   -2.12e+04   -7604.970
x23         2.678e+04   3452.176      7.757      0.000       2e+04    3.35e+04
x24         1.842e+05   3492.486     52.750      0.000    1.77e+05    1.91e+05
x25         1.199e+05   3489.122     34.354      0.000    1.13e+05    1.27e+05
x26         1.581e+04   3508.982      4.504      0.000    8928.246    2.27e+04
x27         2.198e+04   3465.926      6.341      0.000    1.52e+04    2.88e+04
x28         2.974e+04   3459.087      8.597      0.000     2.3e+04    3.65e+04
x29        -2.116e+04   3569.656     -5.928      0.000   -2.82e+04   -1.42e+04
x30         4.076e+04   3463.418     11.767      0.000     3.4e+04    4.75e+04
x31         4.778e+04   3486.541     13.704      0.000    4.09e+04    5.46e+04
x32         3.082e+04   3462.862      8.901      0.000     2.4e+04    3.76e+04
x33         2.948e+04   3453.721      8.535      0.000    2.27e+04    3.62e+04
x34         7.153e+04   3452.964     20.716      0.000    6.48e+04    7.83e+04
x35         4.291e+04   3441.634     12.468      0.000    3.62e+04    4.97e+04
x3

In [33]:
features3 = pd.concat([features2, seattle_housing['WfntLocation']], axis=1)

In [34]:
features3.describe()

,SqFtTotLiving,SqFtLot,Bedrooms,AgeOfBuilding,Garage,x0_1,x0_100,x0_11,x0_12,x0_13,...,x0_81,x0_82,x0_84,x0_87,x0_88,x0_91,x0_92,x0_93,x0_94,WfntLocation
count,30003.000000,3.000300e+04,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,...,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000
mean,2177.319901,1.717131e+04,3.455454,40.914042,352.011199,0.009166,0.006333,0.012265,0.013099,0.011865,...,0.007033,0.011366,0.005733,0.008133,0.013265,0.006799,0.007066,0.013232,0.006199,0.116422
std,1007.939257,5.730894e+04,0.983858,31.848657,284.075036,0.095300,0.079327,0.110070,0.113699,0.108282,...,0.083567,0.106003,0.075499,0.089815,0.114411,0.082179,0.083763,0.114269,0.078493,0.906728
min,3.000000,3.750000e+02,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1450.000000,5.002000e+03,3.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1980.000000,7.500000e+03,3.000000,39.000000,420.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2720.000000,1.066300e+04,4.000000,64.000000,530.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,14980.000000,2.060823e+06,24.000000,118.000000,3450.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000


In [35]:
for i, row in features3.iterrows():
    if row['WfntLocation'] > 0:
        row['WfntLocation'] = 1
    else:
        row['WfntLocation'] = 0

In [36]:
features3_skscaled = ss.fit_transform(features3)

In [37]:
features3['WfntLocation'].value_counts()

0.0    29466
1.0      537
Name: WfntLocation, dtype: int64

In [38]:
model4 = sm.OLS(dep_variable, sm.add_constant(features3_skscaled))
result4 = model4.fit()
result4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.446
Method:                 Least Squares   F-statistic:                     362.0
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:02:51   Log-Likelihood:            -4.4070e+05
No. Observations:               30003   AIC:                         8.815e+05
Df Residuals:                   29935   BIC:                         8.821e+05
Df Model:                          67                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.315e+05   3348.496    248.319      0.000    8.25e+05    8.38e+05
x1          3.545e+05   5276.809     67.185      0.000    3.44e+05    3.65e+05
x2          2.086e+04   3703.597      5.632      0.000    1.36e+04    2.81e+04
x3         -5.121e+04   4438.337    -11.538      0.000   -5.99e+04   -4.25e+04
x4         -4.606e+04   4131.352    -11.150      0.000   -5.42e+04    -3.8e+04
x5         -2.729e+04   4516.407     -6.042      0.000   -3.61e+04   -1.84e+04
x6          1.311e+04   3387.987      3.869      0.000    6468.985    1.98e+04
x7         -1.675e+04   3439.373     -4.871      0.000   -2.35e+04      -1e+04
x8          4.862e+04   3416.747     14.229      0.000    4.19e+04    5.53e+04
x9          7.283e+04   3425.589     21.261      0.000    6.61e+04    7.95e+04
x10         7.456e+04   3428.376     21.748      0.000    6.78e+04    8.13e+04
x11         7.139e+04   3422.465     20.859      0.000    6.47e+04    7.81e+04
x12         4.004e+04   3438.126     11.645      0.000    3.33e+04    4.68e+04
x13         3.213e+04   3415.887      9.405      0.000    2.54e+04    3.88e+04
x14         1.289e+04   3407.559      3.782      0.000    6209.773    1.96e+04
x15         3.758e+04   3416.022     11.001      0.000    3.09e+04    4.43e+04
x16        -1.034e+04   3412.571     -3.031      0.002    -1.7e+04   -3653.196
x17        -1.935e+04   3407.909     -5.679      0.000    -2.6e+04   -1.27e+04
x18        -2.425e+04   3394.736     -7.144      0.000   -3.09e+04   -1.76e+04
x19         4.092e+04   3399.105     12.038      0.000    3.43e+04    4.76e+04
x20        -1.955e+04   3418.459     -5.718      0.000   -2.62e+04   -1.28e+04
x21        -1.522e+04   3382.089     -4.499      0.000   -2.18e+04   -8586.069
x22        -1.393e+04   3400.238     -4.096      0.000   -2.06e+04   -7262.372
x23         2.804e+04   3393.815      8.261      0.000    2.14e+04    3.47e+04
x24         1.804e+05   3435.271     52.512      0.000    1.74e+05    1.87e+05
x25         1.133e+05   3435.979     32.966      0.000    1.07e+05     1.2e+05
x26         1.723e+04   3449.715      4.995      0.000    1.05e+04     2.4e+04
x27         1.918e+04   3408.208      5.627      0.000    1.25e+04    2.59e+04
x28         3.084e+04   3400.554      9.068      0.000    2.42e+04    3.75e+04
x29        -2.249e+04   3509.319     -6.409      0.000   -2.94e+04   -1.56e+04
x30         4.272e+04   3405.182     12.544      0.000     3.6e+04    4.94e+04
x31          5.02e+04   3428.189     14.642      0.000    4.35e+04    5.69e+04
x32         3.198e+04   3404.282      9.393      0.000    2.53e+04    3.86e+04
x33         3.121e+04   3395.534      9.192      0.000    2.46e+04    3.79e+04
x34         6.986e+04   3394.760     20.579      0.000    6.32e+04    7.65e+04
x35         3.174e+04   3400.860      9.333      0.000    2.51e+04    3.84e+04
x3

In [39]:
features4 = pd.concat([features3, seattle_housing['SqFtOpenPorch'], seattle_housing['SqFtEnclosedPorch']], axis =1)

In [40]:
features4['Porch'] = features4['SqFtOpenPorch'] + features4['SqFtEnclosedPorch']

In [41]:
for i, row in features4.iterrows():
    if row['Porch'] > 0:
        row['Porch'] = 1
    else:
        row['Porch'] = 0

In [42]:
features4.describe()

,SqFtTotLiving,SqFtLot,Bedrooms,AgeOfBuilding,Garage,x0_1,x0_100,x0_11,x0_12,x0_13,...,x0_87,x0_88,x0_91,x0_92,x0_93,x0_94,WfntLocation,SqFtOpenPorch,SqFtEnclosedPorch,Porch
count,30003.000000,3.000300e+04,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,...,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000
mean,2177.319901,1.717131e+04,3.455454,40.914042,352.011199,0.009166,0.006333,0.012265,0.013099,0.011865,...,0.008133,0.013265,0.006799,0.007066,0.013232,0.006199,0.017898,80.127087,6.320368,0.553478
std,1007.939257,5.730894e+04,0.983858,31.848657,284.075036,0.095300,0.079327,0.110070,0.113699,0.108282,...,0.089815,0.114411,0.082179,0.083763,0.114269,0.078493,0.132584,131.301997,41.511630,0.497140
min,3.000000,3.750000e+02,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1450.000000,5.002000e+03,3.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1980.000000,7.500000e+03,3.000000,39.000000,420.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.000000,0.000000,1.000000
75%,2720.000000,1.066300e+04,4.000000,64.000000,530.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,110.000000,0.000000,1.000000
max,14980.000000,2.060823e+06,24.000000,118.000000,3450.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2690.000000,2200.000000,1.000000


In [43]:
features4_skscaled = ss.fit_transform(features4)

In [44]:
model5 = sm.OLS(dep_variable, sm.add_constant(features4_skscaled))
result5 = model5.fit()
result5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     350.2
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:02:54   Log-Likelihood:            -4.4063e+05
No. Observations:               30003   AIC:                         8.814e+05
Df Residuals:                   29932   BIC:                         8.820e+05
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.315e+05   3340.620    248.905      0.000    8.25e+05    8.38e+05
x1          3.328e+05   5592.670     59.501      0.000    3.22e+05    3.44e+05
x2          1.746e+04   3719.126      4.695      0.000    1.02e+04    2.47e+04
x3         -4.902e+04   4434.455    -11.054      0.000   -5.77e+04   -4.03e+04
x4         -4.223e+04   4142.365    -10.195      0.000   -5.04e+04   -3.41e+04
x5         -2.588e+04   4511.008     -5.738      0.000   -3.47e+04    -1.7e+04
x6          1.403e+04   3382.098      4.148      0.000    7401.139    2.07e+04
x7         -1.695e+04   3431.563     -4.938      0.000   -2.37e+04   -1.02e+04
x8           5.05e+04   3412.622     14.798      0.000    4.38e+04    5.72e+04
x9          7.359e+04   3418.877     21.526      0.000    6.69e+04    8.03e+04
x10         7.521e+04   3420.939     21.984      0.000    6.85e+04    8.19e+04
x11         7.223e+04   3415.621     21.146      0.000    6.55e+04    7.89e+04
x12         4.112e+04   3431.871     11.983      0.000    3.44e+04    4.79e+04
x13         3.274e+04   3409.263      9.604      0.000    2.61e+04    3.94e+04
x14         1.417e+04   3403.332      4.162      0.000    7494.740    2.08e+04
x15         3.821e+04   3409.438     11.208      0.000    3.15e+04    4.49e+04
x16        -1.009e+04   3404.867     -2.964      0.003   -1.68e+04   -3417.767
x17        -1.911e+04   3400.124     -5.620      0.000   -2.58e+04   -1.24e+04
x18        -2.363e+04   3388.147     -6.975      0.000   -3.03e+04    -1.7e+04
x19          4.19e+04   3392.746     12.351      0.000    3.53e+04    4.86e+04
x20        -2.004e+04   3410.787     -5.876      0.000   -2.67e+04   -1.34e+04
x21        -1.502e+04   3374.726     -4.451      0.000   -2.16e+04   -8407.733
x22        -1.339e+04   3394.462     -3.945      0.000      -2e+04   -6737.291
x23         2.963e+04   3388.902      8.744      0.000     2.3e+04    3.63e+04
x24         1.793e+05   3432.792     52.225      0.000    1.73e+05    1.86e+05
x25          1.13e+05   3429.339     32.951      0.000    1.06e+05     1.2e+05
x26         1.801e+04   3442.710      5.231      0.000    1.13e+04    2.48e+04
x27         2.053e+04   3403.835      6.033      0.000    1.39e+04    2.72e+04
x28         3.189e+04   3394.308      9.395      0.000    2.52e+04    3.85e+04
x29        -2.279e+04   3503.285     -6.507      0.000   -2.97e+04   -1.59e+04
x30         4.258e+04   3397.570     12.533      0.000    3.59e+04    4.92e+04
x31         5.004e+04   3420.613     14.628      0.000    4.33e+04    5.67e+04
x32         3.207e+04   3396.945      9.442      0.000    2.54e+04    3.87e+04
x33         3.201e+04   3389.568      9.444      0.000    2.54e+04    3.87e+04
x34         7.087e+04   3388.936     20.911      0.000    6.42e+04    7.75e+04
x35         3.223e+04   3393.444      9.498      0.000    2.56e+04    3.89e+04
x3

In [45]:
features5 = pd.concat([features4, seattle_housing['TrafficNoise'], seattle_housing['PowerLines'], seattle_housing['OtherNuisances']], axis = 1)

In [46]:
features5.columns

Index(['SqFtTotLiving', 'SqFtLot', 'Bedrooms', 'AgeOfBuilding', 'Garage',
       'x0_1  ', 'x0_100', 'x0_11 ', 'x0_12 ', 'x0_13 ', 'x0_14 ', 'x0_15 ',
       'x0_16 ', 'x0_17 ', 'x0_19 ', 'x0_23 ', 'x0_24 ', 'x0_26 ', 'x0_27 ',
       'x0_28 ', 'x0_29 ', 'x0_30 ', 'x0_31 ', 'x0_33 ', 'x0_34 ', 'x0_35 ',
       'x0_37 ', 'x0_39 ', 'x0_40 ', 'x0_42 ', 'x0_43 ', 'x0_44 ', 'x0_45 ',
       'x0_46 ', 'x0_47 ', 'x0_48 ', 'x0_52 ', 'x0_54 ', 'x0_55 ', 'x0_56 ',
       'x0_58 ', 'x0_59 ', 'x0_6  ', 'x0_62 ', 'x0_63 ', 'x0_64 ', 'x0_65 ',
       'x0_66 ', 'x0_67 ', 'x0_68 ', 'x0_7  ', 'x0_70 ', 'x0_72 ', 'x0_74 ',
       'x0_79 ', 'x0_8  ', 'x0_80 ', 'x0_81 ', 'x0_82 ', 'x0_84 ', 'x0_87 ',
       'x0_88 ', 'x0_91 ', 'x0_92 ', 'x0_93 ', 'x0_94 ', 'WfntLocation',
       'SqFtOpenPorch', 'SqFtEnclosedPorch', 'Porch', 'TrafficNoise',
       'PowerLines', 'OtherNuisances'],
      dtype='object')

In [47]:
features5['Noise'] = np.zeros(len(features5))

In [48]:
# for index, row in features5.iterrows():
#     if (row['TrafficNoise'] > 0): #or (row['PowerLines'] == 'Y') or (row['OtherNuisances'] == 'Y'):
#         row['Noise'] = 1
#     else:
#         row['Noise'] = 0

In [49]:
def clean_noise(row: pd.DataFrame) -> int:
    if (row['TrafficNoise'] > 0) or (row['PowerLines'] == 'Y') or (row['OtherNuisances'] == 'Y'):
        return 1
    else:
        return 0

In [50]:
features5['Noise'] = features5.apply(clean_noise, axis=1)

In [51]:
features5 = features5.drop(['PowerLines', 'OtherNuisances', 'TrafficNoise'], axis=1)

In [52]:
features5_skscaled = ss.fit_transform(features5)

//anaconda3/envs/linreg-env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
//anaconda3/envs/linreg-env/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [53]:
model6 = sm.OLS(dep_variable, sm.add_constant(features5_skscaled))
result6 = model6.fit()
result6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     345.5
Date:                Wed, 04 Dec 2019   Prob (F-statistic):               0.00
Time:                        10:02:59   Log-Likelihood:            -4.4063e+05
No. Observations:               30003   AIC:                         8.814e+05
Df Residuals:                   29931   BIC:                         8.820e+05
Df Model:                          71                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.315e+05   3340.289    248.929      0.000    8.25e+05    8.38e+05
x1          3.325e+05   5592.903     59.455      0.000    3.22e+05    3.43e+05
x2          1.805e+04   3725.420      4.844      0.000    1.07e+04    2.53e+04
x3         -4.912e+04   4434.175    -11.077      0.000   -5.78e+04   -4.04e+04
x4         -4.185e+04   4144.566    -10.096      0.000      -5e+04   -3.37e+04
x5         -2.648e+04   4516.365     -5.864      0.000   -3.53e+04   -1.76e+04
x6          1.419e+04   3382.277      4.194      0.000    7555.951    2.08e+04
x7         -1.708e+04   3431.600     -4.977      0.000   -2.38e+04   -1.04e+04
x8          5.047e+04   3412.303     14.791      0.000    4.38e+04    5.72e+04
x9          7.366e+04   3418.642     21.548      0.000     6.7e+04    8.04e+04
x10         7.549e+04   3422.248     22.057      0.000    6.88e+04    8.22e+04
x11         7.257e+04   3417.770     21.233      0.000    6.59e+04    7.93e+04
x12         4.112e+04   3431.533     11.982      0.000    3.44e+04    4.78e+04
x13         3.302e+04   3410.603      9.683      0.000    2.63e+04    3.97e+04
x14         1.413e+04   3403.021      4.153      0.000    7461.058    2.08e+04
x15         3.837e+04   3409.609     11.253      0.000    3.17e+04    4.51e+04
x16        -9936.6512   3405.038     -2.918      0.004   -1.66e+04   -3262.629
x17        -1.883e+04   3401.399     -5.537      0.000   -2.55e+04   -1.22e+04
x18        -2.369e+04   3387.895     -6.994      0.000   -3.03e+04   -1.71e+04
x19          4.17e+04   3393.265     12.290      0.000    3.51e+04    4.84e+04
x20         -2.01e+04   3410.517     -5.893      0.000   -2.68e+04   -1.34e+04
x21        -1.515e+04   3374.721     -4.488      0.000   -2.18e+04   -8531.717
x22        -1.337e+04   3394.139     -3.938      0.000      -2e+04   -6713.482
x23         2.969e+04   3388.636      8.761      0.000     2.3e+04    3.63e+04
x24         1.795e+05   3433.738     52.282      0.000    1.73e+05    1.86e+05
x25         1.133e+05   3430.711     33.021      0.000    1.07e+05     1.2e+05
x26          1.78e+04   3443.262      5.171      0.000    1.11e+04    2.46e+04
x27         2.051e+04   3403.508      6.027      0.000    1.38e+04    2.72e+04
x28         3.202e+04   3394.356      9.434      0.000    2.54e+04    3.87e+04
x29        -2.284e+04   3502.976     -6.519      0.000   -2.97e+04    -1.6e+04
x30           4.3e+04   3400.971     12.643      0.000    3.63e+04    4.97e+04
x31          5.06e+04   3426.987     14.765      0.000    4.39e+04    5.73e+04
x32         3.241e+04   3398.937      9.534      0.000    2.57e+04    3.91e+04
x33         3.228e+04   3390.763      9.519      0.000    2.56e+04    3.89e+04
x34         7.072e+04   3389.039     20.868      0.000    6.41e+04    7.74e+04
x35         3.253e+04   3395.065      9.582      0.000    2.59e+04    3.92e+04
x3

In [54]:
#variables to add: SaleReason (One Hot Encoded), Stories (may be correlated with TotalSqFt), Condition, 
#SqFtDeck, BuildingGrade (One Hot Encoded), HasAView (of Mt. Rainier, Olympics, etc.), InadequateParking (1 for 
#inadequate, 2 for adequate), Duplex Homes vs. Single Family Homes (need to target cost per sq ft instead of 
#overall sale price), Lot size to home proportion

#Each person make a DataFrame that contains the feature (with OHE if appropriate) from the original seattle_housing
#DataFrame. We will iteratively concat these new features to the model, seeing if they are correlated with other 
#features and doing appropriate scaling

In [55]:
#Wyatt - SaleReason (OHE), HasAView, Lot size to Home Proportion

In [278]:
SaleReason = seattle_housing[['SaleReason']]
SaleReason_trans = ohe.fit_transform(SaleReason)
SaleReasonOHE = pd.DataFrame(SaleReason_trans.todense(), columns = ohe.get_feature_names())
SaleReasonOHE = SaleReasonOHE.add_suffix('SaleReason')

In [267]:
views = seattle_housing[['MtRainier', 'Olympics', 'Cascades', 'Territorial', 'SeattleSkyline', 'PugetSound', 
                        'LakeWashington', 'LakeSammamish', 'SmallLakeRiverCreek', 'OtherView']]

,MtRainier,Olympics,Cascades,Territorial,SeattleSkyline,PugetSound,LakeWashington,LakeSammamish,SmallLakeRiverCreek,OtherView
count,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000,30003.000000
mean,0.028031,0.060661,0.065527,0.235976,0.021165,0.062294,0.060694,0.014365,0.024631,0.012265
std,0.281618,0.404692,0.404098,0.770284,0.242174,0.410943,0.403574,0.207744,0.269484,0.177897
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [268]:
def clean_view(row: pd.DataFrame) -> int:
    if sum(row) > 0:
        return 1
    else:
        return 0

In [274]:
views['View'] = views.apply(clean_view, axis = 1)
Views = views[['View']]

/opt/anaconda3/envs/linreg-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [272]:
LotProportion = ((seattle_housing["SqFt1stFloor"] + seattle_housing["SqFtGarageAttached"]) / seattle_housing["SqFtLot"])

In [ ]:
#Akash - Stories, BuildingGrade (OHE), InadequateParking (OHE)

In [ ]:
features9 = pd.concat([features5, seattle_housing['Stories']], axis = 1)
features9_skscaled = ss.fit_transform(features9)
model10 = sm.OLS(dep_variable, sm.add_constant(features9_skscaled))
result10 = model10.fit()
result10.summary()


In [ ]:
ohe = OneHotEncoder(categories='auto')
buildinggrade_trans = ohe.fit_transform(seattle_housing[['BldgGrade']])
buildinggrade = pd.DataFrame(buildinggrade_trans.todense(), columns = ohe.get_feature_names())
features10 = pd.concat([features9, buildinggrade], axis=1)
features10.drop(['x0_3.0'], axis = 1)
features10_skscaled = ss.fit_transform(features10)
model11 = sm.OLS(dep_variable, sm.add_constant(features10_skscaled))
result11 = model11.fit()
result11.summary()

In [ ]:
ohe = OneHotEncoder(categories='auto')
parking_trans = ohe.fit_transform(seattle_housing[['InadequateParking']])
parking = pd.DataFrame(parking_trans.todense(), columns = ohe.get_feature_names())
features11 = pd.concat([features10, parking], axis=1)
features11.drop(['x0_1.0'], axis = 1)
features11_skscaled = ss.fit_transform(features11)
model12 = sm.OLS(dep_variable, sm.add_constant(features11_skscaled))
result12 = model12.fit()
result12.summary()

In [ ]:
#Hunter - Condition, SqFtDeck, Is Home duplex or single?

In [75]:
# Condition DataFrame:
#Make total number of bathrooms column - maybe consider multiplying by 0.5/0.75
condition_df = seattle_housing[['Condition']]
display(condition_df.head())

#Perform One-Hot Encoding on the DF
condition_ohe_df = ohe.fit_transform(condition_df[['Condition']])
condition_ohe_df = pd.DataFrame(condition_ohe_df.todense(), columns = ohe.get_feature_names())
condition_ohe_df.columns = ['Condition: Poor', 'Condition: Fair', 'Condition: Average', 'Condition: Good', 'Condition: Very Good']
print(condition_ohe_df.shape)
display(condition_ohe_df.head())

,Condition
0,4.0
1,3.0
2,5.0
3,4.0
4,3.0


(30003, 5)


,Condition: Poor,Condition: Fair,Condition: Average,Condition: Good,Condition: Very Good
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0


In [74]:
# SqFtDeck DataFrame:
sq_ft_deck_df = seattle_housing[['SqFtDeck']]
print(sq_ft_deck_df.shape)
display(sq_ft_deck_df.head())

(30003, 1)


,SqFtDeck
0,0.0
1,270.0
2,0.0
3,0.0
4,180.0


In [66]:
# #DuplexOrSingle DataFrame:
# duplex_or_single_df = seattle_housing[['HBUAsIfVacant']]

# # SqFtDeck DataFrame:
# sq_ft_deck_df = seattle_housing[['SqFtDeck']]
# display(sq_ft_deck_df.head())


# #Perform One-Hot Encoding on the DF
# sq_ft_deck_ohe_df = ohe.fit_transform(sq_ft_deck_df[['SqFtDeck']])
# sq_ft_deck_ohe_df = pd.DataFrame(sq_ft_deck_ohe_df.todense(), columns = ohe.get_feature_names())

# print(sq_ft_deck_ohe_df.shape)
# display(sq_ft_deck_ohe_df.head())